In [8]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
import math
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, mean_squared_error
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
import gensim
import string
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import CuDNNLSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
import re
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
import warnings
# from keras import backend as K
warnings.filterwarnings("ignore")

In [2]:
#!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.8 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/Users/zhumingjia/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


# Load data

In [9]:
df = pd.read_csv('toxic_classification.csv', index_col=0)
df

,target,comment_text
0,0,"This is so cool. It's like, 'would you want yo..."
1,0,Thank you!! This would make my life a lot less...
2,0,This is such an urgent design problem; kudos t...
3,0,Is this something I'll be able to install on m...
4,1,haha you guys are a bunch of losers.
...,...,...
1804869,0,"Maybe the tax on ""things"" would be collected w..."
1804870,0,What do you call people who STILL think the di...
1804871,0,"thank you ,,,right or wrong,,, i am following ..."
1804872,1,Anyone who is quoted as having the following e...


# Preprocessing

In [10]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))
def preprocess(text_string):
    text_string = text_string.lower() 
    text_string = re.sub('[^A-Za-z0-9]+', ' ', text_string) 
    
    x = text_string.split()
    new_text = []
    
    for word in x:
        if word not in stop_words:
            new_text.append(stemmer.stem(word))
            
    text_string = ' '.join(new_text)
    return text_string

In [11]:
df['preprocessed_text'] = df['comment_text'].apply(preprocess)

In [35]:
df_0 = df[df['target'] == 0].iloc[:20000, :]
df_1 = df[df['target'] == 1].iloc[:20000, :]
df_small = pd.concat([df_0, df_1])

In [60]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(df_small['preprocessed_text'], df_small['target'], test_size=0.2, random_state=0)


# LSTM

In [61]:
dataset = list(df_small['preprocessed_text'])
vocab_set = set()
tokenizer = nltk.tokenize.word_tokenize

for sample in dataset:
    words = tokenizer(sample)
    vocab_set.update(words)

# Number of unique words
num_unique_words = len(vocab_set)
num_unique_words

30123

In [62]:
list_length = [len(i.split(" ")) for i in dataset]
max(list_length)

221

In [63]:
# Hyperparameters of the model
vocab_size = 30123 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 221 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [68]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 221, 100)          3012300   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 128)               84480     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 24)                3096      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 25        
Total params: 3,099,901
Trainable params: 3,099,901
Non-trainable params: 0
_________________________________________________________________


In [69]:
num_epochs = 2
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
#print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

Epoch 1/2
900/900 [==============================] - 78s 83ms/step - loss: 0.5072 - accuracy: 0.7363 - val_loss: 0.3011 - val_accuracy: 0.8778
Epoch 2/2
900/900 [==============================] - 72s 80ms/step - loss: 0.2030 - accuracy: 0.9232 - val_loss: 0.2944 - val_accuracy: 0.8794


# Results

In [72]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
f1_score(y_test,pred_labels)

0.8881859106947316

In [73]:
precision_score(y_test,pred_labels)

0.8643144683874023

In [74]:
recall_score(y_test,pred_labels)

0.9134134134134134

In [75]:
training_indices = X_train.index
test_indices = X_test.index

In [88]:
wrong_idx_fp = []
wrong_idx_fn = []
y_test_list = list(y_test)
for i in range(len(y_test)):
    if y_test_list[i] != pred_labels[i]:
        if pred_labels[i] == 1:
            wrong_idx_fp.append(i)
        else:
            wrong_idx_fn.append(i)

In [89]:
len(wrong_idx_fp)

573

In [90]:
len(wrong_idx_fn)

346

In [107]:
test_text = list(X_test)

In [138]:
def get_text(idx):
    t = test_text[wrong_idx_fp[idx]]
    print(t)
    print("=========")
    pre_t = list(df_small[df_small['preprocessed_text'] == t]['comment_text'])[0]
    print(pre_t)

In [143]:
def get_text_fn(idx):
    t = test_text[wrong_idx_fn[idx]]
    print(t)
    print("=========")
    pre_t = list(df_small[df_small['preprocessed_text'] == t]['comment_text'])[0]
    print(pre_t)

In [140]:
get_text(0)

vote lesser two evil life sick find attitud voter refus vote lesser two evil unsophist except condescend one might counter say continu support two parti system altern either dupe part dupe class suspect republican parti fractur soon three candid year trump clinton sander republican question one third parti candid
I have been voting for the lesser of two evils for most of my life, and I am sick of it.

I find your attitude that voters who refuse to vote for the lesser of two evils as "unsophisticated" to be exceptionally condescending.  One might counter by saying those who continue to support the two-party system over all other alternatives are either being duped, or are part of the duping class.

 I suspect the Republican party will fracture soon and we will have three candidates this year: Trump, Clinton or Sanders, and a Republican. The question is, which one is the third-party candidate?


In [110]:
get_text(1)

Phil Knight, his Nike-rich buddies, and sycophants such as our esteemed legislators, will tell you what is "best for you."  Follow their "lead", stay out of their way, get "behind" them  and, if you're lucky, a few bucks might fall out of their pockets for you to grovel for.  

Face it, the very wealthy know "what is best".......for them.....you don't really count.  They tell us to    support an elite IAAF meet which will disrupt the University.....those damn students are just "in the way"; will cost over ten million dollars per day; will financially benefit only a very few individuals (those who don't really need the money); will have prohibitive ticket prices (so mere peasants can't attend); and will cost about 25 million in tax dollars.....gosh, how could we possibly turn that down??? 

In my opinion, our local Democatic legislators have, once again,  "sold out" to the wealthy special interest groups.  I've been an active Democrat for over fifty years and find your actions sad...


In [111]:
get_text(3)

I'd be more supportive of Miss Loretta if she'd said "why are there more Blacks in the system in the first place?" And "the frequency of the use of force in our jails is unacceptable no matter what color offenders' skin is". 

Instead it sounds like the Oscars; everyone talkin bout boycotting due to the lack of "diversity" and honoring more "people of color", when really they're advocating only for Blacks only, not Hispanics/Latinos, or Asians, or Natives, who are the more discriminated against by far.


In [120]:
get_text(12)

Good point, but still had nothing to do with liberals or liberalism.


In [144]:
get_text_fn(1)

everi bit boast self righteous obama
He's every bit as boastful and self righteous as Obama!


In [145]:
get_text_fn(3)

would burn hous ground
I would have burned the house to the ground.


In [149]:
get_text_fn(7)

predict pedant
You are so predictable and pedantic.
